In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
fname = "test_fold_one"
basepath = "./-paraphrase-multilingual-MiniLM-L12-v2_3015"

## Load data

In [3]:
import pandas as pd

def loadData(fname, basepath):
    sample = pd.read_csv(f"{basepath}/{fname}.csv")
    sample['Document'] = sample.apply(lambda row: ': '.join(row.astype(str)), axis=1)
    return sample

## Question Generation

In [4]:
from qa_generation import QAGenerator
from prompt_factory import PromptFactory
from tqdm.notebook import tqdm

template = '''
        Gedraag je als een {role} die online informatie zoekt.
        Patiënten stellen meestal vragen als:
{{
    "qa_list": [
        {{
            "question": "Hoe kan ik een verstopte voedingssonde doorspoelen?",
            "answer": "Om een verstopte voedingssonde door te spoelen, kunt u de volgende stappen proberen: 1. Sluit de spuit rechtstreeks aan op de sonde (niet op het voedingssysteem). Als er een verstopping is in het voedingssysteem, kunt u dit vervangen. 2. Neem een spuit van 10 cc en spuit met lichte druk lauwwarm water door de voedingssonde. Herhaal dit zo nodig nog een keer. 3. Als het oplossen van de verstopping niet lukt, laat dan lauwwarm water 30 minuten inwerken en herhaal de bovenstaande procedure nogmaals. 4. Als u de verstopping kunt zien, kunt u proberen om de voedingssonde op die plek zachtjes te kneden. Als de verstopping dan loskomt, kunt u het doorspuiten met lauwwarm water. Belangrijk: Gebruik nooit een voerdraad of koolzuurhoudend bronwater of frisdranken, omdat dit kan leiden tot perforatie. Daarnaast wordt het afraden om natriumbicarbonaat te gebruiken als medicatie de oorzaak is van de verstopping, omdat dit de verstopping groter kan maken."
        }}
    ]
}}


        in het json formaat: 
        {format}

Document Informatie:
        {document}

        Belangrijke Opmerking: Bij het genereren van vragen, gebruik specifieke termen en benamingen uit het document in plaats van algemene termen zoals 'dit onderzoek' of 'die procedure'. Verwijs direct naar de procedure of het document met de exacte naam om nauwkeurigheid en duidelijkheid in de vragen te waarborgen. Vermijd algemeenheden en zorg ervoor dat elke vraag direct gerelateerd is aan de verstrekte documentinformatie.


        Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
        {format}

        Zorg ervoor dat elke vraag en antwoord paar in een geldig JSON-formaat is. Dit betekent dat vragen en antwoorden tussen dubbele aanhalingstekens moeten staan, en de algemene structuur moet overeenkomen met het vereiste JSON-schema.
        '''

roles = [
    'Patiënt',
    'Nieuwe Ouder',
    'Oudere Patiënt',
    'Persoon die een Tweede Mening Zoekt',
    'Reiziger die Medisch Advies Na Reizen Zoekt',
    'Zorgverlener die Informatie Zoekt']

prompt_factory = PromptFactory(prompt=template,roles=roles)

num_questions_per_doc = 5

QAGenerator = QAGenerator()
# Generate the data
import pandas as pd
from tqdm import tqdm


def generateQuestions(df):
    data = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0],desc="Generating QAs"):
        doc = row["Document"]
        # Do something with col1 and col2
        try:
            for i in range(2):
                qa_list = QAGenerator.generate_qas(prompt_factory.generate_prompt(doc, num_questions_per_doc))
                for qa in qa_list:
                    new_row = row.to_dict()
                    new_row['Question'] = qa.question
                    new_row['Answer'] = qa.answer
                    data.append(new_row)

        except Exception as e:
                print(f"Failed to generate QA for doc: {doc[:100]}. Error: {e}")
                continue
    if(len(data) == 0):
       return pd.DataFrame()
    result = pd.DataFrame(data)
    result = result.dropna(subset=['Document', 'Question', 'Answer'])
    result = result[(result['Document'] != '') & (result['Question'] != '') & (result['Answer'] != '')]
    return result


In [5]:
from fact_qa import FactQAGenerator
from prompt_factory import PromptFactory
import langchain

template = '''
Gedraag je als iemand die online informatie zoekt en stelt vragen gebaseerd op specifieke feiten zoals locatie, telefoonnummer, naam, enzovoort.

Patiënten of gebruikers kunnen bijvoorbeeld vragen stellen als:
{{
    "qa_list": [
        {{
            "question": "Wat is het telefoonnummer van de kliniek voor noodgevallen?",
            "answer": "Het telefoonnummer van de kliniek voor noodgevallen is 012-345-6789."
        }},
        {{
            "question": "Waar is de hoofdingang van het ziekenhuis gelegen?",
            "answer": "De hoofdingang van het ziekenhuis is gelegen aan de Eerste Gezondheidsstraat 123 in Amsterdam."
        }}
        // meer vragen en antwoorden gebaseerd op specifieke feiten
    ]
}}

in het json formaat:
{format}

Document Informatie:
{document}

Belangrijke Opmerking: Bij het genereren van vragen, gebruik specifieke termen en benamingen uit het document in plaats van algemene termen. Verwijs direct naar de specifieke feit (zoals locatie, telefoonnummer, naam) om nauwkeurigheid en duidelijkheid in de vragen te waarborgen. Vermijd algemeenheden.

Maak vragen die beantwoord kunnen worden met de volgende informatie: {fact}

Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
{format}

Zorg ervoor dat elke vraag en antwoord paar in een geldig JSON-formaat is. Dit betekent dat vragen en antwoorden tussen dubbele aanhalingstekens moeten staan, en de algemene structuur moet overeenkomen met het vereiste JSON-schema.
        '''

base_prompt = langchain.PromptTemplate(
            template=template,
            input_variables=['fact', 'n', 'document', 'format']
        )

num_questions_per_doc = 5

FactQAGenerator = FactQAGenerator()

def generateFactQuestions(df):
    data = []
    # Generate the data
    for index, row in tqdm(df.iterrows(), total=df.shape[0],desc="Generating Fact QAs"):
        doc = row['Document']
        # Assuming phone numbers are separated by commas
        facts = row['Context Entities']
        try:
            for i in range(2):
                for entity in extract_desired_entities(facts)[:5]:
                    fact = entity["value"]
                    prompt = base_prompt.partial(
                        n=num_questions_per_doc,
                        document=doc)
                    try:
                        qa_list = FactQAGenerator.generate_question_for_fact(prompt, fact)
                        for qa in qa_list:
                            new_row = row.to_dict()
                            new_row['Question'] = qa.question
                            new_row['Answer'] = qa.answer
                            new_row['Fact'] = entity
                            print(entity)
                            data.append(new_row)
                    except:
                        pass
        except Exception as e:
                print(f"Failed to generate QA for document: {doc[:20]}. Error: {e:20}")

    if(len(data) == 0):
        return pd.DataFrame()

    result = pd.DataFrame(data)
    result = result.dropna(subset=['Document', 'Question', 'Answer'])
    result = result[(result['Document'] != '') & (result['Question'] != '') & (result['Answer'] != '')]
    return result

In [6]:
def update_df_with_lambda(df, column_name, lambda_function):
     # Check if the column does not exist in the DataFrame, if not, initialize it with NA
    if column_name not in df.columns:
        df[column_name] = pd.NA
    
    # Apply the lambda function only to rows where the column value is NA
    df[column_name] = df.apply(lambda row: lambda_function(row) if pd.isna(row[column_name]) else row[column_name], axis=1)
    return df

In [7]:
from answer_context_filter import calculate_bleu_score, calculate_rouge_score

def calculate_scores(df):
    # Calculate BLEU and ROUGE scores
    #df['Question-Context BLEU'] = df.apply(lambda row: calculate_bleu_score(row['Document'], row['Question']), axis=1)
    #df['Answer-Context BLEU'] = df.apply(lambda row: calculate_bleu_score(row['Document'], row['Answer']), axis=1)

    #df['Question-Context ROUGE'] = df.apply(lambda row: calculate_rouge_score(row['Document'], row['Question']), axis=1)
    #df['Answer-Context ROUGE'] = df.apply(lambda row: calculate_rouge_score(row['Document'], row['Answer']), axis=1)
    #df['Question-Context ROUGE-L F1'] = df['Question-Context ROUGE'].apply(lambda x: x[0]['rouge-l']['f'] if x else None)
    #df['Answer-Context ROUGE-L F1'] = df['Answer-Context ROUGE'].apply(lambda x: x[0]['rouge-l']['f'] if x else None)

    df = update_df_with_lambda(
        df, 'Question-Context ROUGE',
        lambda row: calculate_rouge_score(row['Document'], row['Question']) if pd.isna(row.get('Question-Context ROUGE')) else row['Question-Context ROUGE']
    )

    # Update 'Answer-Context ROUGE' only if it is NA
    df = update_df_with_lambda(
        df, 'Answer-Context ROUGE',
        lambda row: calculate_rouge_score(row['Document'], row['Answer']) if pd.isna(row.get('Answer-Context ROUGE')) else row['Answer-Context ROUGE']
    )

    # Update 'Question-Context ROUGE-L F1' based on the 'Question-Context ROUGE' column, only if it is NA
    df = update_df_with_lambda(
        df, 'Question-Context ROUGE-L F1',
        lambda row: row['Question-Context ROUGE'][0]['rouge-l']['f'] if pd.isna(row.get('Question-Context ROUGE-L F1')) and row['Question-Context ROUGE'] else None
    )

    # Update 'Answer-Context ROUGE-L F1' based on the 'Answer-Context ROUGE' column, only if it is NA
    df = update_df_with_lambda(
        df, 'Answer-Context ROUGE-L F1',
        lambda row: row['Answer-Context ROUGE'][0]['rouge-l']['f'] if pd.isna(row.get('Answer-Context ROUGE-L F1')) and row['Answer-Context ROUGE'] else None
    )

    return df


def rougeFilter(df):
    df = calculate_scores(df)
    df = df[df['Answer-Context ROUGE-L F1'] > 0.1]
    return df

## Vector filter

In [8]:
from vector_filter import precompute_embeddings
from vector_filter import filter_dataframe

def vectorFilter(df):
    df["Embedding"] = precompute_embeddings(df, 'Question')
    df = filter_dataframe(df, threshold=0.9)
    return df

## Entity Filter

In [9]:
def contains_all_elements(list1, list2):
    """
    Check if list1 contains all elements of list2.
    Each element is a dictionary with 'entityType' and 'value' keys.
    """
    # Convert each dictionary in the lists to a tuple (entityType, value) for easy comparison
    #set1 = {tuple(d.items()) for d in list1}
    #set2 = {tuple(d.items()) for d in list2}
    set1 = {d['value'] for d in list1 if 'value' in d}
    set2 = {d['value'] for d in list2 if 'value' in d}

    # Check if every element in set2 is also in set1
    return set2.issubset(set1)

def check_entity_match(row):
    return contains_all_elements(row['Context Entities'], row['Answer Entities'])

# Apply the function to each row to create the new column


In [10]:
from extract import extract_all_entities

def extractEntities(df):
    df['Context Entities'] = df['Document'].apply(extract_all_entities)
    df['Question Entities'] = df['Question'].apply(extract_all_entities)
    df['Answer Entities'] = df['Answer'].apply(extract_all_entities)
    df['Entity Match'] = df.apply(check_entity_match, axis=1)
    """  
    df = update_df_with_lambda(df, 'Context Entities', 
                               lambda row: extract_all_entities(row['Document']) 
                                if row.get('Context Entities') is None else row['Context Entities'])

    # Update or create 'Question Entities' column by applying entity extraction function if the value is NA
    df = update_df_with_lambda(df, 'Question Entities', 
                               lambda row: extract_all_entities(row['Question']) 
                               if row.get('Question Entities')is None else row['Question Entities'])

    # Update or create 'Answer Entities' column by applying entity extraction function if the value is NA
    df = update_df_with_lambda(df, 'Answer Entities', 
                               lambda row: extract_all_entities(row['Answer']) 
                               if row.get('Answer Entities') is None else row['Answer Entities'])

    # Update or create 'Entity Match' column by applying the check entity match function if the value is NA
    df = update_df_with_lambda(df, 'Entity Match', 
                               lambda row: check_entity_match(row) 
                               if row.get('Entity Match') is None else row['Entity Match'])
    """
    return df



/home/jovyan/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2024-01-18 14:20:18.609526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 14:20:18.738034: I tensorflow/core/util/port.cc:104] oneDNN custom operations are 

2024-01-18 14:20:35,268 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-PER, S-LOC, B-MISC, E-MISC, B-ORG, E-ORG, I-ORG, I-PER, B-LOC, I-LOC, E-LOC, I-MISC, <START>, <STOP>


In [11]:
def contains_desired_entity(entities_list):
    desired_types = {'Phone', 'Email'}
    return any(entity['entityType'] in desired_types for entity in entities_list)


def filterFactQuestions(df):
    df = extractEntities(df)
    return df[~((df['Question'].str.split().str.len() < 30) & df['Answer Entities'].apply(contains_desired_entity))]

def extract_desired_entities(entities_list):
    # Define the desired entity types
    desired_types = {'Phone', 'Email'}
    
    # Extract entities that match the desired types
    matching_entities = [entity for entity in entities_list if entity['entityType'] in desired_types]
    
    return matching_entities

In [12]:
def filterFactQuestionCorrectness(df):
    df['Answer Entities'] = df['Answer'].apply(extract_all_entities)
    # Apply a filter condition to each row
    #filtered_df = df[df.apply(lambda row: row["Fact"] in row["Answer Entities"], axis=1)]
    filtered_df = df[df.apply(lambda row: any( row["Fact"]['value'] == entity['value'] for entity in row["Answer Entities"]) if isinstance(row["Answer Entities"], list) else False, axis=1)]
    return filtered_df

#### LLM Scoring

In [13]:
from llm_filter import estimate_relevance

def llmScore(df):
    return update_df_with_lambda(df, 'LLM Score', lambda row: estimate_relevance(row['Question'], row['Document']))

# Run

In [14]:
import os

sample = loadData(fname, basepath)

df = pd.DataFrame(columns = sample.columns)



for i in tqdm(range(5), desc="Processing"):
    raw_file_path = f"{basepath}/output_raw_{i}_{fname}.csv"

    if os.path.exists(raw_file_path):
        # Load the file into a DataFrame if it exists
        print(f"Skipping generation: {i}")
        new_questions_df = pd.read_csv(raw_file_path)
    else:
        df_gen = sample[~sample['Document'].isin(df['Document'])]
        new_questions_df = generateQuestions(df_gen)
        if df_gen.empty:
            break
    new_questions_df.to_csv(raw_file_path)
    df = pd.concat([df, new_questions_df], ignore_index=True, sort=False)
    df = rougeFilter(df)
    df = vectorFilter(df)
    df = filterFactQuestions(df)
    df = llmScore(df)


## take top one for each doc
idx = df.groupby('Document')['LLM Score'].idxmax()
df = df.loc[idx]

df.to_csv(f"{basepath}/output_full_{fname}.csv")
df.to_csv(f"{basepath}/output_{fname}.csv", columns = ['topic_id', 'filename', 'header', 'section_text', 'Question', 'Answer'])


Generating QAs:   6%|▌         | 3/50 [00:40<10:06, 12.90s/it]

Failed to generate QA for doc: 55: Gemalen voeding bij slikklachten: Vragen: Heeft u na het lezen van deze folder nog vragen? Neem . Error: 'list' object has no attribute 'qa_list'



Generating QAs:  80%|████████  | 40/50 [10:24<02:37, 15.70s/it]

Failed to generate QA for doc: 3: Geestelijke verzorging, ondersteuning en begeleiding van nierpatiënten: Kennismaken: Indien u heb. Error: 'list' object has no attribute 'qa_list'



Processing:  40%|████      | 2/5 [20:43<27:19, 546.34s/it]   

...

Processing:  60%|██████    | 3/5 [22:08<11:11, 335.64s/it]

Skipping generation: 3


Processing:  80%|████████  | 4/5 [23:29<03:55, 235.05s/it]

Skipping generation: 4


Processing: 100%|██████████| 5/5 [25:01<00:00, 300.29s/it]


## Export Q&A Pairs

In [15]:
df.to_csv(f"{basepath}/output_full_{fname}.csv")
df.to_csv(f"{basepath}/output_{fname}.csv", columns = ['topic_id', 'filename', 'header', 'section_text', 'Question', 'Answer'])

In [16]:
df

,topic_id,filename,header,section_text,Document,Question,Answer,Question-Context ROUGE,Answer-Context ROUGE,Question-Context ROUGE-L F1,Answer-Context ROUGE-L F1,Embedding,Context Entities,Question Entities,Answer Entities,Entity Match,LLM Score,Unnamed: 0
40,0,Afdeling 5 west (ZGT Almelo),Belangrijke informatie,Bij problemen en vragen kunt u in eerste insta...,0: Afdeling 5 west (ZGT Almelo): Belangrijke i...,Wanneer is de beste tijd om de verantwoordelij...,Het wordt aangeraden om de verantwoordelijke v...,"[{'rouge-1': {'r': 0.1016949152542373, 'p': 0....","[{'rouge-1': {'r': 0.3220338983050847, 'p': 0....",0.054795,0.325581,"[0.015346979065916389, -0.02335966420286628, 0...","[{'entityType': 'ORG', 'value': 'ZGT Almelo'}]",[],[],True,90.0,NaN
97,0,Anesthesie kinderen (POS),Vragen en contact,Neem dan gerust contact op met ZGT polikliniek...,0: Anesthesie kinderen (POS): Vragen en contac...,Op welke dagen en tijden kan ik bellen met ZGT...,U kunt bellen met ZGT polikliniek preoperatiev...,"[{'rouge-1': {'r': 0.14814814814814814, 'p': 0...","[{'rouge-1': {'r': 0.42592592592592593, 'p': 1...",0.235294,0.597403,"[0.0015574133475772627, -0.02143312744446034, ...","[{'entityType': 'Phone', 'value': '+3188708539...","[{'entityType': 'ORG', 'value': 'ZGT'}]","[{'entityType': 'ORG', 'value': 'ZGT'}]",True,100.0,NaN
158,0,"CVA revalidatie, revalidatie na een herseninfa...",Nuttige adressen en patiëntenverenigingen,De Nederlandse CVA-vereniging “Samen verder” P...,"0: CVA revalidatie, revalidatie na een herseni...","Wat is het bezoekadres van Roessingh, centrum ...","Het bezoekadres van Roessingh, centrum voor re...","[{'rouge-1': {'r': 0.025477707006369428, 'p': ...","[{'rouge-1': {'r': 0.05732484076433121, 'p': 0...",0.048193,0.105882,"[0.02015472410168831, -0.015820374872564914, 0...","[{'entityType': 'Phone', 'value': '+3188383830...","[{'entityType': 'LOC', 'value': 'Roessingh'}]","[{'entityType': 'LOC', 'value': 'Roessingh'}, ...",True,80.0,NaN
154,0,Cardiologische nazorgpoli,Heeft u vragen?,Voor vragen zijn wij op werkdagen van 8.00 - 1...,0: Cardiologische nazorgpoli: Heeft u vragen?:...,Wat is het verzoek bij het sturen van een e-mail?,Vermeld in de e-mail s.v.p. uw achternaam en g...,"[{'rouge-1': {'r': 0.023809523809523808, 'p': ...","[{'rouge-1': {'r': 0.2619047619047619, 'p': 1....",0.039216,0.415094,"[0.0010312061294578145, -0.034699813326347945,...","[{'entityType': 'Phone', 'value': '+3188708330...",[],[],True,100.0,NaN
95,0,Follow-up Borstkliniek Oost-Nederland,Afspraak maken,Wanneer het tijd is voor uw controlebezoek kri...,0: Follow-up Borstkliniek Oost-Nederland: Afsp...,Wanneer ontvang ik een uitnodiging voor mijn c...,U ontvangt een uitnodiging van ons wanneer het...,"[{'rouge-1': {'r': 0.2, 'p': 0.5, 'f': 0.28571...","[{'rouge-1': {'r': 0.5, 'p': 0.769230769230769...",0.214286,0.363636,"[-0.002944429192951676, -0.03932623592909306, ...","[{'entityType': 'ORG', 'value': 'Borstkliniek ...",[],[],True,100.0,NaN
93,0,Gezonde moeder of rooming-in ouder,Afdeling moeder en kind (verloskunde),"Ziekenhuislocatie Almelo, telefoonnummer 088 7...",0: Gezonde moeder of rooming-in ouder: Afdelin...,Welke afdeling is verantwoordelijk voor verlos...,De afdeling moeder en kind is verantwoordelijk...,"[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, '...","[{'rouge-1': {'r': 0.17647058823529413, 'p': 0...",0.000000,0.206897,"[0.0262017725345024, -0.02287935922128619, 0.0...","[{'entityType': 'Phone', 'value': '+3188708516...",[],[],True,100.0,NaN
88,0,Madentherapie,Vragen,Heeft u naar aanleiding van deze folder vragen...,0: Madentherapie: Vragen: Heeft u naar aanleid...,Wat is het belang van dagelijks contact met de...,Het dagelijks contact met de polikliniek chiru...,"[{'rouge-1': {'r': 0.14705882352941177, 'p': 0...","[{'rouge-1': {'r': 0.14705882352941177, 'p': 0...",0.222222,0.192308,"[0.018920464964401564, -0.0025052338134383878,...","[{'entityType': 'Phone', 'value': '+3188708524...",[],[],True,100.0,NaN
91,0,"Schisis, sluiting

# Fact Q&A

In [17]:
sample = loadData(fname, basepath)

sample['Context Entities'] = sample['Document'].apply(extract_all_entities)

new_questions_df = generateFactQuestions(sample)
new_questions_df.to_csv(f"{basepath}/output_fact_raw_{fname}.csv")
df = vectorFilter(new_questions_df)
df = llmScore(df)
df = filterFactQuestionCorrectness(df)

## take top one for each doc
df['Fact Value'] = df['Fact'].apply(lambda x: x.get('value'))
idx = df.groupby(['Document', 'Fact Value'])['LLM Score'].idxmax()
df = df.loc[idx]

Generating Fact QAs:   0%|          | 0/50 [00:00<?, ?it/s]

{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}


Generating Fact QAs:   6%|▌         | 3/50 [00:27<07:07,  9.09s/it]

{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Phone', 'value': '+31887085243'}
{'entityType': 'Phone', 'value': '+31887085243'}
{'entityType': 'Phone', 'value': '+31887085243'}
{'entityType': 'Phone', 'value': '+31887085243'}
{'entityType': 'Phone', 'value': '+31887085243'}


Generating Fact QAs:  54%|█████▍    | 27/50 [00:36<00:24,  1.07s/it]

...

Generating Fact QAs:  64%|██████▍   | 32/50 [02:37<03:24, 11.35s/it]

{'entityType': 'Phone', 'value': '+31887083210'}
{'entityType': 'Phone', 'value': '+31887083210'}
{'entityType': 'Phone', 'value': '+31887083210'}
{'entityType': 'Phone', 'value': '+31887083210'}
{'entityType': 'Phone', 'value': '+31887083210'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}
{'entityType': 'Phone', 'value': '+31887085396'}


Generating Fact QAs:  66%|██████▌   | 33/50 [03:04<03:47, 13.36s/it]

{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31887084228'}
{'entityType': 'Phone', 'value': '+31883838300'}
{'entityType': 'Phone', 'value': '+31883838300'}
{'entityType': 'Phone', 'value': '+31883838300'}
{'entityType': 'Phone', 'value': '+31883838300'}
{'entityType': 'Phone', 'value': '+31883838300'}
{'entityType': 'Phone', 'value': '+319003000300'}
{'entityType': 'Phone', 'value': '+319003000300'}
{'entityType': 'Phone', 'value': '+319003000300'}
{'entityType': 'Phone', 'value': '+319003000300'}
{'entityType': 'Phone', 'value': '+319003000300'}
{'entityType': 'Phone', 'value': '+31263512512'}
{'entityType': 'Phone', 'value': '+31263512512'}
{'entityType': 'Phone', 'value': '+31263512512'}
{'entityType': 'Phone', 'value': '+31263512512'}
{'entityType': 'Phone', 'value': '+31263512512'}
{'entityType': 

Generating Fact QAs: 100%|██████████| 50/50 [04:26<00:00,  5.33s/it]


In [18]:

df.to_csv(f"{basepath}/output_full_fact_{fname}.csv")
df.to_csv(f"{basepath}/output_fact_{fname}.csv", columns = ['topic_id', 'filename', 'header', 'section_text', 'Fact', 'Question', 'Answer'])